In [7]:
import pandas as pd
import pymongo
import sys

In [8]:
# Provide the mongodb atlas url to connect python to mongodb using pymongo
CONNECTION_STRING = "mongodb+srv://dbuser:<password>@cluster0.wq2aw.mongodb.net/myFirstDatabase?retryWrites=true&w=majority"
# Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
client = pymongo.MongoClient(CONNECTION_STRING,connect=False)

NameError: name 'sys' is not defined